In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re
import sys
sys.path.append('googlereviewscraper')
# Assuming you've defined DriverInit and ReviewScraper in .py files in the same directory as your notebook
from googlereviewscraper.driver_init import DriverInit
from googlereviewscraper.review_scraper import ReviewScraper


In [2]:
url = 'https://www.google.com/search?rlz=1C1GCEA_enHR1085HR1085&q=Casablanca%20Reviews&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxI2sTQzMjUzsDC2NDUzMTMzN7W03MDI-IpRyDmxODEpJzEvOVEhKLUsM7W8eBErFkEAULFNy0QAAAA&rldimm=4962560839564667599&tbm=lcl&hl=en&sa=X&ved=0CAYQ5foLahcKEwiAzYbJzIKEAxUAAAAAHQAAAAAQCg&biw=1536&bih=729&dpr=1.25#lkt=LocalPoiReviews&arid=ChZDSUhNMG9nS0VJQ0FnSURSNWJlRUhnEAE'

In [3]:
driver_init.close_browser()

NameError: name 'driver_init' is not defined

In [4]:
driver_init = DriverInit()
driver_init.navigate_to_url(url)

In [5]:
driver_init.navigate_to_reviews_tab()

In [6]:
review_scraper = ReviewScraper(driver_init)

In [7]:
review_scraper.load_reviews()

TimeoutException: Message: 
